## SQLAlcemy動作確認

In [6]:
from flask import Flask
from modules.models.models import *
from app import create_app
from modules import spotify_tools as st


In [7]:
# db start
app = create_app()
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///..\\..\\data\\user_database.db"
# pytest
# app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///data\\user_database.db"
db.init_app(app)
app.app_context().push()


In [8]:
db.create_all()

2024-03-02 20:01:44,529 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:01:44,531 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-03-02 20:01:44,532 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:01:44,532 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fav_artists")
2024-03-02 20:01:44,532 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:01:44,532 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sim_users")
2024-03-02 20:01:44,532 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:01:44,546 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
# test
user_list = UserList()
# add
user_list.add_user(user_token="#thisistoken", display_name="username", pass_word="user") 
assert db.session.query(User).filter_by(user_token="#thisistoken").first(), "Userオブジェクトが作成されていません"
# delete



2024-03-02 20:01:57,627 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:01:57,630 INFO sqlalchemy.engine.Engine INSERT INTO users (user_token, user_name, pass_word, done) VALUES (?, ?, ?, ?)
2024-03-02 20:01:57,631 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ('#thisistoken', 'username', 'user', 0)
2024-03-02 20:01:57,634 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: users.mail_address
[SQL: INSERT INTO users (user_token, user_name, pass_word, done) VALUES (?, ?, ?, ?)]
[parameters: ('#thisistoken', 'username', 'user', 0)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [10]:

db.session.commit()
db.session.close()



PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) NOT NULL constraint failed: users.mail_address
[SQL: INSERT INTO users (user_token, user_name, pass_word, done) VALUES (?, ?, ?, ?)]
[parameters: ('#thisistoken', 'username', 'user', 0)]
(Background on this error at: https://sqlalche.me/e/20/gkpj) (Background on this error at: https://sqlalche.me/e/20/7s2a)